# Software Environment and Cloud Computing

[![Virus on Linux](fig/virus.png)](https://www.gnu.org/fun/jokes/evilmalware.en.html)

In modern computational science, running code is only half the
challenge.
The other half is managing the software environment it depends on.
Different projects may require different compilers, libraries, or
Python packages, and the ability to reproduce results across HPC
systems, laptops, and cloud platforms is important.

This week, we will learn practical tools for creating and managing
software environments:
* Package managers (e.g., `apt-get`) for installing software at the
  system level
* Building from source when precompiled packages are unavailable or
  when you do not have root access to the system.
* HPC `module` for loading different versions of software on a shared
  system.
* Python virtual environments for dependency isolation
* Containers (i.e., Docker) for reproducibility and portability

[![Dependency](fig/dependency.png)](https://xkcd.com/2347/)

## Software Management Basics

On Unix/Linux systems, most software is installed through package
managers, which automatically handle downloading, installing, and
configuring programs and their dependencies.

Common package managers include:
* Debian/Ubuntu: [`apt`](https://wiki.debian.org/Apt) (or the older `apt-get`)
* Red Hat/Fedora/CentOS: [`yum`](https://fedoraproject.org/wiki/Yum) or [`dnf`](https://docs.fedoraproject.org/en-US/quick-docs/dnf/)
* Arch Linux: [`pacman`](https://wiki.archlinux.org/title/Pacman)
* macOS: [`brew`](https://brew.sh/) (Homebrew) and [`port`](https://www.macports.org/) (MacPorts)

[![sudo](fig/sandwich.png)](https://xkcd.com/149/)

By default, package managers install software system-wide, which
requires root (administrator) privileges.
On personal machines, this is often done by prefixing commands with
`sudo` (i.e., "superuser do").

On shared systems such as HPC clusters, users typically do not have
root access, so alternative approaches are used (modules, virtual
environments, containers).

In this lab, we will use Docker to create a sandbox environment where
you can safely practice system-level package management commands as if
you had root.
We will cover Docker in more detail later, but for now you can start a
sandbox with:

```bash
docker run -it --rm ubuntu
```

[![Python Environment](fig/python_environment.png)](https://xkcd.com/1987/)